<a href="https://colab.research.google.com/github/sayanbanerjee32/keyword_search/blob/main/Semantic_search_BERT_FAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download data

In [12]:
%%capture
# To import kaggle datasets
!pip install --upgrade pip 
!pip install kaggle==1.5.6
!pip install datasets
!pip install transformers 

In [2]:
# # import colab libraries
# from google.colab import files
# # import kaggle json to connect to kaggle user account to download datsets
# files.upload()

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so lets move it there.
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/Kaggle_api_auth/kaggle.json ~/.kaggle/

In [4]:
# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
# download our dataset
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews -p 'datasets'

 19% 5.00M/25.7M [00:00<00:00, 39.9MB/s]
100% 25.7M/25.7M [00:00<00:00, 128MB/s] 


In [6]:
#unzip
!unzip datasets/imdb-dataset-of-50k-movie-reviews.zip -d datasets/

Archive:  datasets/imdb-dataset-of-50k-movie-reviews.zip
  inflating: datasets/IMDB Dataset.csv  


## Sentence BERT
https://huggingface.co/course/chapter5/6?fw=tf

In [9]:
import re
def fix_text(txt):
    return re.sub(r'(<br />)+',' ',txt)
    # return clean(txt,
    #             fix_unicode=True,               # fix various unicode errors
    #             to_ascii=True,                  # transliterate to closest ASCII representation
    #             lower=True,                     # lowercase text
    #             no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
    #             no_urls=False,                  # replace all URLs with a special token
    #             no_emails=True,                # replace all email addresses with a special token
    #             no_phone_numbers=True,         # replace all phone numbers with a special token
    #             no_numbers=True,               # replace all numbers with a special token
    #             no_digits=False,                # replace all digits with a special token
    #             no_currency_symbols=True,      # replace all currency symbols with a special token
    #             no_punct=True,                 # remove punctuations
    #             replace_with_punct="<PUNCT>",          # instead of removing punctuations you may replace them
    #             replace_with_url="<URL>",
    #             replace_with_email="<EMAIL>",
    #             replace_with_phone_number="<PHONE>",
    #             replace_with_number="<NUMBER>",
    #             replace_with_digit="0",
    #             replace_with_currency_symbol="<CUR>",
    #             lang="en"                       # set to 'de' for German special handling
    #         )

In [1]:
import pandas as pd
df = pd.read_csv('datasets/IMDB Dataset.csv')
df.shape


(50000, 2)

In [10]:
df['review'] = df['review'].apply(fix_text)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming te...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [11]:
from datasets import Dataset

imdb_dataset = Dataset.from_pandas(df)
imdb_dataset

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 50000
})

## Create word embedding

In [3]:
from transformers import AutoTokenizer, TFAutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = TFAutoModel.from_pretrained(model_ckpt, from_pt=True)

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFMPNetModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFMPNetModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFMPNetModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFMPNetModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMPNetModel for predictions without further training.


In [5]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="tf"
    )
    encoded_input = {k: v for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [12]:
embedding = get_embeddings(imdb_dataset["review"][0])
embedding.shape

TensorShape([1, 768])

In [ ]:
embeddings_dataset = imdb_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["review"]).numpy()[0]}
)

INFO:tensorflow:Assets written to: ram://20fba5ec-2e40-4443-b04a-b36e1bbdb987/assets


INFO:tensorflow:Assets written to: ram://20fba5ec-2e40-4443-b04a-b36e1bbdb987/assets


0ex [00:00, ?ex/s]

### FAISS for similarity search

In [ ]:
embeddings_dataset.add_faiss_index(column="embeddings")

In [ ]:
question = "How can I load a dataset offline?"
question_embedding = get_embeddings([question]).numpy()
question_embedding.shape

In [ ]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

In [ ]:
samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [ ]:
for _, row in samples_df.iterrows():
    print(f"Review: {row.review}")
    print("=" * 50)
    print()